In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

# import networkx as nx
# from networkx.algorithms import bipartite
import igraph as ig

import pycountry_convert

import matplotlib.pyplot as plt

# Data preparation

In [4]:
projects = pd.read_csv('../data/raw/cordis-HORIZONprojects-csv/csv/project.csv', delimiter=';')
projects.head()

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi
0,101095387,AISym4MED,SIGNED,Synthetic and scalable data platform for medic...,2022-12-01,2026-11-30,6341765,6341765,HORIZON.2.1,HORIZON-HLTH-2022-IND-13-02,2022-11-22,HORIZON,HORIZON-HLTH-2022-IND-13,HORIZON-HLTH-2022-IND-13,RIA,NaN,AISym4Med aims at developing a platform that w...,2022-12-30 17:42:45,244110,10.3030/101095387
1,101062897,IMPROVE,SIGNED,IMaging PROstate cancer using ViscoElastic bio...,2023-09-01,2025-08-31,0,"181152,96",HORIZON.1.2,HORIZON-MSCA-2021-PF-01-01,2022-06-30,HORIZON,HORIZON-MSCA-2021-PF-01,HORIZON-MSCA-2021-PF-01,MSCA-PF,NaN,IMPROVE aims to demonstrate the potentiality o...,2022-08-26 02:24:17,240147,10.3030/101062897
2,101044731,SAINTBARTH,SIGNED,"Slavery, Abolition and Archipelagic Connection...",2024-01-01,2028-12-31,1992925,1992925,HORIZON.1.1,ERC-2021-COG,2022-07-11,HORIZON,ERC-2021-COG,ERC-2021-COG,HORIZON-AG,NaN,Surveys and datasets of the trans-Atlantic sla...,2022-08-26 14:59:15,240177,10.3030/101044731
3,101065876,TEMP-MOMO,SIGNED,Drivers of spatiotemporal variation in tempera...,2022-09-01,2024-08-31,0,"195914,88",HORIZON.1.2,HORIZON-MSCA-2021-PF-01-01,2022-07-13,HORIZON,HORIZON-MSCA-2021-PF-01,HORIZON-MSCA-2021-PF-01,HORIZON-AG-UN,NaN,Europe emerges as a major climate warming hots...,2022-08-26 15:30:46,240278,10.3030/101065876
4,101057389,CINDERELLA,SIGNED,Clinical Validation of an AI-based approach to...,2022-06-01,2026-05-31,5379050,5378299,HORIZON.2.1,HORIZON-HLTH-2021-DISEASE-04-04,2022-05-23,HORIZON,HORIZON-HLTH-2021-DISEASE-04,HORIZON-HLTH-2021-DISEASE-04,RIA,NaN,Breast cancer is the most commonly diagnosed c...,2022-09-04 13:43:33,241461,10.3030/101057389


In [5]:
organisations = pd.read_csv('../data/raw/cordis-HORIZONprojects-csv/csv/organization.csv', delimiter=';')
organisations.head()

,projectID,projectAcronym,organisationID,vatNumber,name,shortName,SME,activityType,street,postCode,city,country,nutsCode,geolocation,organizationURL,contactForm,contentUpdateDate,rcn,order,role,ecContribution,netEcContribution,totalCost,endOfParticipation,active
0,101095387,AISym4MED,908114874,BE0692797754,ZABALA BRUSSELS,ZABALA BRUSSELS,True,PRC,RUE BELLIARD 20,1040,BRUXELLES,BE,NaN,"50.840092,4.3814743",NaN,https://ec.europa.eu/info/funding-tenders/oppo...,2022-12-30 17:42:45,1954294,5,participant,464375.0,464375.0,464375,False,NaN
1,101095387,AISym4MED,900973055,FI29362646,SAIDOT OY,SAIDOT,True,PRC,TEKNIIKANTIE 2,02150,Espoo,FI,NaN,"60.1668875,24.7488936",NaN,https://ec.europa.eu/info/funding-tenders/oppo...,2022-12-30 17:42:45,1965161,11,participant,548750.0,548750.0,548750,False,NaN
2,101095387,AISym4MED,991228063,BE0890217005,TIME.LEX,time.lex,True,PRC,JOSEPH STEVENSSTRAAT 7,1000,BRUSSEL,BE,NaN,"50.8413225,4.3524748",NaN,https://ec.europa.eu/info/funding-tenders/oppo...,2022-12-30 17:42:45,2349568,7,participant,417187.5,417187.5,"417187,5",False,NaN
3,101095387,AISym4MED,999993468,GB649926678,IMPERIAL COLLEGE OF SCIENCE TECHNOLOGY AND MED...,Imperial,False,HES,SOUTH KENSINGTON CAMPUS EXHIBITION ROAD,SW7 2AZ,LONDON,UK,NaN,"51.5073219,-0.1276473",http://www.imperial.ac.uk,https://ec.europa.eu/info/funding-tenders/oppo...,2022-12-30 17:42:45,1905566,14,associatedPartner,NaN,0.0,NaN,False,NaN
4,101095387,AISym4MED,974142483,ESA50086412,INSTRUMENTACION Y COMPONENTES SA,NaN,False,PRC,CALLE ALAUN 8 PLA ZA,50197,ZARAGOZA,ES,NaN,"41.6410997,-0.9956188",www.inycom.es,https://ec.europa.eu/info/funding-tenders/oppo...,2022-12-30 17:42:45,2054577,2,participant,1205000.0,1205000.0,1205000,False,NaN


# Data analysis

In [17]:
organisations.columns

Index(['projectID', 'projectAcronym', 'organisationID', 'vatNumber', 'name',
       'shortName', 'SME', 'activityType', 'street', 'postCode', 'city',
       'country', 'nutsCode', 'geolocation', 'organizationURL', 'contactForm',
       'contentUpdateDate', 'rcn', 'order', 'role', 'ecContribution',
       'netEcContribution', 'totalCost', 'endOfParticipation', 'active'],
      dtype='object')

In [26]:
organisations.groupby('organisationID')['projectID'].count().sort_values(ascending=False)

organisationID
999997930    385
999984059    238
999991722    215
999979500    188
999991334    178
            ... 
909600041      1
909604794      1
909606734      1
909617986      1
936473600      1
Name: projectID, Length: 14393, dtype: int64

In [46]:
organisations[['organisationID', 'name', 'projectID', 'projectAcronym']].values

array([[908114874, 'ZABALA BRUSSELS', 101095387, 'AISym4MED'],
       [900973055, 'SAIDOT OY', 101095387, 'AISym4MED'],
       [991228063, 'TIME.LEX', 101095387, 'AISym4MED'],
       ...,
       [999990946, 'KUNGLIGA TEKNISKA HOEGSKOLAN', 101045453, 'fitsCAN'],
       [999522921, 'REGION STOCKHOLM', 101045453, 'fitsCAN'],
       [969818320,
        'NATIONAL SCIENCE CENTER KHARKOV INSTITUTE OF PHYSICS AND TECHNOLOGY',
        101072693, 'UAinEuratom21']], dtype=object)

In [51]:
G = ig.Graph.TupleList(
      edges=organisations[['organisationID', 'projectID']].values,
      directed=False
      )

In [52]:
is_bipartite, types = G.is_bipartite(return_types = True)
is_bipartite

True

In [53]:
types[0]

False

In [55]:
G.vs[0]

igraph.Vertex(<igraph.Graph object at 0x1429d6040>, 0, {'name': 908114874})

In [56]:
G_org_projection = G.bipartite_projection(types=types, which=False)

In [57]:
G_org_projection.summary()

'IGRAPH UNW- 14393 347142 -- \n+ attr: name (v), weight (e)'

In [58]:
G_org_projection.is_connected()

False

In [62]:
components = G_org_projection.connected_components()

In [64]:
H = G_org_projection.connected_components().giant()

In [65]:
H.summary()

'IGRAPH UNW- 14028 347103 -- \n+ attr: name (v), weight (e)'

In [66]:
H.vcount() / G.vcount() * 100

69.60059538576036

In [67]:
G.vs[0]

igraph.Vertex(<igraph.Graph object at 0x1429d6040>, 0, {'name': 908114874})